#### 분석 환경 설정 

In [13]:
# 사용 모듈 불러오기
import os
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup as BS
from selenium import webdriver 
from functools import reduce

# 경로 설정
os.chdir(r"C:\Users\GilseungAhn\Desktop\한양대\3. 프로젝트\2. 진행중인 과제\포스트코로나 AI 챌린지\데이터\2. 외부데이터")

#### 해외유입확진자 수 수집

In [14]:
# 메인 url 설정 및 수집 결과 초기화
main_url = 'https://www.cdc.go.kr'
board_page_num = 1
result_dic = dict() # key: 날짜, value: 누적해외유입확진자 수

while True:
    print("탐색중인 게시판 번호: {}".format(board_page_num))
    board_url = 'board.es?mid=a20501000000&bid=0015&nPage=' + str(board_page_num) # board_page_num번째 게시판 목록의 url
    
    # board_url에 해당하는 HTML 가져오기
    board_response = requests.get(main_url + '/' + board_url)
    board_text = board_response.text
    
    # 게시판에 등록된 열 개의 문서 가운데, 코로나 관련 기사가 한 건도 나오지 않으면, 탐색을 중지
    if not('코로나바이러스감염증-19 국내 발생 현황' in board_text or '신종코로나바이러스감염증 국내 발생 현황' in board_text):
        break
    
    # 게시판에 등장한 코로나 관련 기사의 링크 수집
    content_tag_list = BS(board_text).find_all("li", "title")[1:]
    for content_tag in content_tag_list:
        content_no = str(content_tag.find('a')['href']).split('list_no=')[1].split('&')[0] # 코로나 관련 기사 링크의 번호 (유니크한 ID 역할)
        content_url = 'https://www.cdc.go.kr/board/board.es?mid=a20501000000&bid=0015&act=view&list_no=' + content_no
        
        # 코로나 관련 기사 가져오기
        content_response = requests.get(content_url)                
        content_text = content_response.text
        
        # 코로나 관련 기사에서 해외유입 정보 가져오기 및 저장
        content_title = str(BS(content_text).find("meta", attrs = {"name":"title"}))
        
        # 코로나바이러스감염증-19 국내 발생 현황 혹은 신종코로나바이러스감염증 국내 발생 현황이 문서 제목에 포함된 경우만 크롤링
        if '코로나바이러스감염증-19 국내 발생 현황' in content_title or '신종코로나바이러스감염증 국내 발생 현황' in content_title:
            
            # 정례브리핑이 문서 제목에 포함되고, 해외유입이 문서 내용에 포함된 경우만 크롤링
            if '정례브리핑' in content_title:
                content_date = content_title.split('(')[1].split(',')[0]
                if '해외유입' in content_text:
                    try:                        
                        # 수집 문구: ..., 총 누적 확진자수는 10,752명(해외유입 1,056명) ...에서 해외유입 **명
                        content_num_definite_diagnosis = int(content_text.split("해외유입 ")[1].split('명')[0].replace(',', '')) # 천 명을 넘는 순간부터 콤마(,)가 붙어서 콤마를 제거
                    except:
                        content_num_definite_diagnosis = 0
                
                # 수집 결과 저장
                if content_date not in result_dic.keys():
                    result_dic[content_date] = content_num_definite_diagnosis                
                time.sleep(1.5)                
    board_page_num += 1

# 수집한 데이터 자료형 변환 및 저장
def convert_string_to_datetime(s): # M월 D일
    year = 2020
    month = int(s.split('월')[0])
    day = int(s.split(' ')[1].split('일')[0])
    return pd.Timestamp(year = year, month = month, day = day)

# 수집 결과를 데이터 프레임 형태로 변환
result_df = pd.DataFrame(result_dic.items(), columns = ['일자', '누적확진자'])

# 일자 변수의 타입 변환 (string -> datetime)
result_df['일자'] = result_df['일자'].apply(convert_string_to_datetime)

# 일자 기준 정렬 및 데이터 내보내기 (일자별_누적해외유입확진자수.csv)
result_df.sort_values(by = '일자', inplace = True)
result_df.to_csv("일자별_누적해외유입확진자수.csv", index = False, encoding = "utf8")

탐색중인 게시판 번호: 1
탐색중인 게시판 번호: 2
탐색중인 게시판 번호: 3
탐색중인 게시판 번호: 4
탐색중인 게시판 번호: 5
탐색중인 게시판 번호: 6
탐색중인 게시판 번호: 7
탐색중인 게시판 번호: 8
탐색중인 게시판 번호: 9
탐색중인 게시판 번호: 10
탐색중인 게시판 번호: 11
탐색중인 게시판 번호: 12
탐색중인 게시판 번호: 13
탐색중인 게시판 번호: 14
탐색중인 게시판 번호: 15
탐색중인 게시판 번호: 16
탐색중인 게시판 번호: 17
탐색중인 게시판 번호: 18
탐색중인 게시판 번호: 19
탐색중인 게시판 번호: 20
탐색중인 게시판 번호: 21
탐색중인 게시판 번호: 22
탐색중인 게시판 번호: 23
탐색중인 게시판 번호: 24
탐색중인 게시판 번호: 25
탐색중인 게시판 번호: 26
탐색중인 게시판 번호: 27
탐색중인 게시판 번호: 28
탐색중인 게시판 번호: 29
탐색중인 게시판 번호: 30


#### 해외증시 수집

In [15]:
# webdriver 구동하기 (반드시 chromedriver.exe가 설치되어 있어야 함)
driver = webdriver.Chrome(r'C:\Users\GilseungAhn\Desktop\기타\chromedriver_win32\chromedriver.exe')

# 수집대상 url 정의 및 수집 데이터 초기화
url_dict = {"니케이":"https://finance.naver.com/world/sise.nhn?symbol=NII@NI225#",
           "다우산업":"https://finance.naver.com/world/sise.nhn?symbol=DJI@DJI",
           "S&P500":"https://finance.naver.com/world/sise.nhn?symbol=SPI@SPX",
           "상해종합": "https://finance.naver.com/world/sise.nhn?symbol=SHS@000001"}

# 결과를 저장할 리스트 (요소: 해외증시 데이터 (DataFrame))
dfList = []

for (url_id, url) in url_dict.items():
    # url별 수집 데이터를 저장하는 사전 초기화 
    result_dict = dict()
    
    # url 가져오기
    driver.get(url)
    
    # 현재 날짜 기준 최대 max_page 페이지까지 수집 
    max_page = 10 # 10페이지 이내에 분석 범위(2020년) 내 데이터가 모두 들어오므로 max_page를 10페이지로 설정
    
    for page_id in range(1, max_page + 1):
        print("해외증시:{}, 페이지번호:{}".format(url_id, page_id))
        time.sleep(1.5)
        
        # 표 번호 (1 ~ max_page) 클릭
        driver.find_element_by_id("dayLink" + str(page_id)).click()
        time.sleep(1.5)
        
        # dayTable: 해외증시가 있는 HTML 테이블
        table = driver.find_element_by_id("dayTable")
        
        # dayTable의 행을 기준으로 데이터 수집
        for tr in table.find_elements_by_tag_name("tr"):
            try: # 해외 증시의 경우에는 공백(' ')으로 데이터가 구분되어 있음. 공백이 없는 데이터는 해외 증시 데이터가 아님.
                date, value = tr.text.split(' ')[:2]                   
                result_dict[date] = value       
            except: 
                pass
        
    # 증시별 수집 결과 저장 및 dfList에 추가
    result_df = pd.DataFrame(result_dict.items(), columns = ['일자', url_id + "지수"])
    result_df[url_id + "지수"] = result_df[url_id + "지수"].str.replace(',', '').astype(float)
    dfList.append(result_df)

# 해외증시 데이터 병합 (단, 지수별로 장이 안 열리는 날은 결측이 되어 있음)
output_df = reduce(lambda x, y: pd.merge(x, y, on = '일자', how = 'outer'), dfList)    

# 일자 변수의 타입 변환 (string -> datetime)
output_df['일자'] = pd.to_datetime(output_df['일자'])    

# 일자 기준 정렬 및 데이터 내보내기 (일자별_해외지수.csv)
output_df.sort_values(by = '일자', inplace = True)
output_df.to_csv("일자별_해외지수.csv", index = False, encoding = "utf8")

해외증시:니케이, 페이지번호:1
해외증시:니케이, 페이지번호:2
해외증시:니케이, 페이지번호:3
해외증시:니케이, 페이지번호:4
해외증시:니케이, 페이지번호:5
해외증시:니케이, 페이지번호:6
해외증시:니케이, 페이지번호:7
해외증시:니케이, 페이지번호:8
해외증시:니케이, 페이지번호:9
해외증시:니케이, 페이지번호:10
해외증시:다우산업, 페이지번호:1
해외증시:다우산업, 페이지번호:2
해외증시:다우산업, 페이지번호:3
해외증시:다우산업, 페이지번호:4
해외증시:다우산업, 페이지번호:5
해외증시:다우산업, 페이지번호:6
해외증시:다우산업, 페이지번호:7
해외증시:다우산업, 페이지번호:8
해외증시:다우산업, 페이지번호:9
해외증시:다우산업, 페이지번호:10
해외증시:S&P500, 페이지번호:1
해외증시:S&P500, 페이지번호:2
해외증시:S&P500, 페이지번호:3
해외증시:S&P500, 페이지번호:4
해외증시:S&P500, 페이지번호:5
해외증시:S&P500, 페이지번호:6
해외증시:S&P500, 페이지번호:7
해외증시:S&P500, 페이지번호:8
해외증시:S&P500, 페이지번호:9
해외증시:S&P500, 페이지번호:10
해외증시:상해종합, 페이지번호:1
해외증시:상해종합, 페이지번호:2
해외증시:상해종합, 페이지번호:3
해외증시:상해종합, 페이지번호:4
해외증시:상해종합, 페이지번호:5
해외증시:상해종합, 페이지번호:6
해외증시:상해종합, 페이지번호:7
해외증시:상해종합, 페이지번호:8
해외증시:상해종합, 페이지번호:9
해외증시:상해종합, 페이지번호:10
